# 策略持仓权重管理
---

In [1]:
# 设置 Redis 链接信息，用于存储持仓权重信息
import os
os.environ['RWC_REDIS_URL'] = 'redis://***:***@***:6379/0'

# 也可以使用 dotenv 来设置环境变量
from dotenv import load_dotenv
# load_dotenv(r'path/to/.env', override=True)
load_dotenv(r"D:\ZB\git_repo\waditu\czsc\examples\test_offline\.env", override=True)

True

### 一、写入策略持仓权重

策略持仓权重样例数据可以从飞书文档中下载：

In [ ]:
import czsc
import redis
import pandas as pd

# 创建 BlockingConnectionPool 连接池，必须设置 decode_responses=True 参数，否则返回的数据是 bytes 类型
connection_pool = redis.BlockingConnectionPool.from_url(os.getenv("RWC_REDIS_URL"), decode_responses=True)


# 策略写入阶段，必须设置 send_heartbeat=True 参数，这样可以在写入数据时自动更新策略的最新写入时间
rwc = czsc.RedisWeightsClient('TestStrategy', connection_pool=connection_pool, key_prefix='Weights', send_heartbeat=True)

# 首次写入，建议设置一些策略元数据
rwc.set_metadata(description='测试策略：仅用于读写redis测试', base_freq='日线', author='测试A', outsample_sdt='20220101')

# 查看策略元数据
print("策略元数据：", rwc.metadata)

# 写入策略持仓权重，样例数据下载：https://s0cqcxuy3p.feishu.cn/wiki/Pf1fw1woQi4iJikbKJmcYToznxb
dfw = pd.read_feather(r"C:\Users\zengb\Downloads\weight_example.feather")

# on_bar 模式下可以按单条数据写入
rwc.publish(**dfw.iloc[0].to_dict())

# 批量写入可以提高效率
rwc.publish_dataframe(dfw, overwrite=False, batch_size=10000)